# Teste da importação dos modelos e variaveis categoricas

In [19]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder

In [20]:
with open("../api/models/best_model.pkl", "rb") as f:
    best_model = pickle.load(f)

In [21]:
with open("../api/models/encoders.pkl", "rb") as f:
    encoders = pickle.load(f)

In [22]:
symptoms_encoder = encoders["symptoms"]
sex_encoder = encoders["sex"]
nature_encoder = encoders["nature"]
treatment_encoder = encoders["treatment"]
disease_encoder = encoders["disease"]

In [23]:
def prever_tratamento(novos_sintomas, idade, sexo, natureza, tratamento):
    # Verificar se os novos valores estão nos encoders
    for sintoma in novos_sintomas:
        if sintoma not in symptoms_encoder.classes_:
            raise ValueError(f"Sintoma não reconhecido: {sintoma}")
    if sexo not in sex_encoder.classes_:
        raise ValueError(f"Sexo não reconhecido: {sexo}")
    if natureza not in nature_encoder.classes_:
        raise ValueError(f"Natureza não reconhecida: {natureza}")
    if tratamento not in treatment_encoder.classes_:
        raise ValueError(f"Tratamento não reconhecido: {tratamento}")

    # Codificar os sintomas
    sintomas_codificados = symptoms_encoder.transform(novos_sintomas)

    # Calcular a moda dos sintomas codificados
    sintoma_moda = max(set(sintomas_codificados), key=list(sintomas_codificados).count)

    # Preparar os dados de entrada
    novos_dados = pd.DataFrame(
        {
            "Symptoms": [sintoma_moda],
            "Age": [idade],
            "Sex": [sexo],
            "Nature": [natureza],
            "Treatment": [tratamento],
        }
    )

    # Codificar os dados de entrada
    novos_dados["Sex"] = sex_encoder.transform(novos_dados["Sex"])
    novos_dados["Nature"] = nature_encoder.transform(novos_dados["Nature"])
    novos_dados["Treatment"] = treatment_encoder.transform(novos_dados["Treatment"])

    # Prever o tratamento
    predicao = best_model.predict(novos_dados)
    tratamento_previsto = disease_encoder.inverse_transform(predicao)

    return tratamento_previsto[0]

In [24]:
sintomas = ["Cough"]
idade = 5.0
sexo = "female"
natureza = "high"
tratamento = "Omalizumab"
tratamento = prever_tratamento(sintomas, idade, sexo, natureza, tratamento)
print(f"O tratamento previsto para os sintomas {sintomas} é: {tratamento}")

O tratamento previsto para os sintomas ['Cough'] é: Asthma
